In [0]:
!pip install --upgrade keras
!pip install gdown
!pip install tensorflow-gpu==1.15

In [0]:
%cd /content/drive/My Drive/person_detection/keras-retinanet

!pip install .
!python setup.py build_ext --inplace
%cd ../../../../../

In [0]:
import keras
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
from google.colab.patches import cv2_imshow
import tensorflow as tf
import pandas as pd

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

keras.backend.tensorflow_backend.set_session(get_session())

model_path = '/content/drive/My Drive/person_detection/keras-retinanet/snapshots/13-04-2020_103546_resnet152_13_0.h5'    
model = models.load_model(model_path, backbone_name='resnet152')
model = models.convert_model(model)
labels_to_names = {0: 'pedestrian'}                                     

video_path = '/content/drive/My Drive/person_detection/bus_showcase.avi'  
output_path = '/content/drive/My Drive/person_detection/bus_showcase_detected.mp4' 
fps = 20





# Creating the new feature dataframe with detections


In [0]:
vcapture = cv2.VideoCapture(video_path)

width = int(vcapture.get(cv2.CAP_PROP_FRAME_WIDTH))  # uses given video width and height
height = int(vcapture.get(cv2.CAP_PROP_FRAME_HEIGHT))
vwriter = cv2.VideoWriter(output_path,cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height)) #
print('width: ', width, 'height: ', height)
num_frames = int(vcapture.get(cv2.CAP_PROP_FRAME_COUNT))
min_side_factor = 2 

def run_detection_video(video_path):
    downscale_factor_y = 2
    df_detections = create_zeroed_df(downscale_factor_y, num_frames)

    frame_index = 0
    success = True
    start = time.time()

    while success:
        if frame_index % 100 == 0:
            print("frame: ", frame_index)
        frame_index += 1
        # Read next image
        success, image = vcapture.read()

        if success:

            draw = image.copy()
            draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

            image = preprocess_image(image)
            image, scale = resize_image(image, min_side=width * min_side_factor, max_side=1333)
            boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
            boxes /= scale

            for box, score, label in zip(boxes[0], scores[0], labels[0]):
                # scores are sorted so we can break
                if score < 0.35:
                    break

                color = label_color(label)

                b = box.astype(int)
                draw_box(draw, b, color=color)

                #fill df with probability at the center of y axis, consider rezizing with min_side_factor

                df_detections.iloc[frame_index, int((b[3] + b[1]) / min_side_factor / 2 )] = score

                caption = "{} {:.3f}".format(labels_to_names[label], score)
                draw_caption(draw, b, caption)
            detected_frame = cv2.cvtColor(draw, cv2.COLOR_RGB2BGR)
            vwriter.write(detected_frame)  # overwrites video slice

    vcapture.release()
    vwriter.release()  
    end = time.time()
    df_detections.to_csv('video_detection.csv', header=None)
    print("Total Time: ", end - start)

run_detection_video(video_path)


In [0]:
def create_zeroed_df(factor_y, num_frames):
  return pd.DataFrame(np.zeros((num_frames, int(height / factor_y))))
